In [ ]:
from pyspark.sql.types import *
import json
from pyspark.sql.functions import *
# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
#jsonSchema = StructType([StructField("name", StringType(), True) ])
jsonSchema = StructType([ StructField("last_reported", TimestampType(), True), StructField("name", StringType(), True) ])

# Strem DataFrame representing data in the JSON files
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9985) \
    .schema(jsonSchema) \
    .load()

stream_objects = lines.select(from_json(lines.value, jsonSchema).alias("stream_object"))
stream_objects.isStreaming

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
    stream_objects
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("append")  # complete = all the counts should be in the table
    .start()
)

In [ ]:
#query.awaitTermination()

In [ ]:
#spark.sql("select * from counts").toJSON().collect()

In [ ]:
# RUN a simple webservice on default port 5000 to return the in-memory table as is
from flask import Flask

app = Flask(__name__)

# route to access data
@app.route("/")
def hello():
        rdd_json = spark.sql("select * from counts").toJSON().collect()
    return json.dumps(rdd_json)

if __name__ == "__main__":
    app.run()